In [1]:
import pandas as pd
import ioh

In [2]:
def run_for_dimension(dimension):
    all_bbob_optimums=[]
    for problem_id in range (1,25):
        for instance_id in range (1,1000):
            f = ioh.get_problem(problem_id,instance_id,dimension)
            o = f.optimum.x
            all_bbob_optimums+=[tuple([problem_id,instance_id,dimension]+list(f.optimum.x)+ [f.optimum.y])]
    columns=['problem_id','instance_id','dimension']+[f'x_{i}' for i in range (0,dimension)]+['y']
    all_bbob_optimums=pd.DataFrame(all_bbob_optimums, columns=columns)
    all_bbob_optimums.to_csv(f'algorithm_run_data/BBOB_optimums_dim_{dimension}.csv', index=False)

In [3]:
for dimension in [3,5,10,20]:
    run_for_dimension(dimension)

In [ ]:
all_minimums_df=pd.DataFrame()
for dimension in [3,5,10,20]:
    for seed in [200,400,600,800,1000]:
        for algorithm_name in ['DE','ES','PSO']:
            df = pd.read_csv(f'algorithm_run_data/{algorithm_name}_dim_{dimension}_seed_{seed}.csv', index_col=[0])
            grouped = df.groupby(['algorithm_name','problem_id','instance_id','iteration']).min()[str(dimension)].to_frame().reset_index()

            for iteration in range(0,50):
                minimums_up_to_iteration=grouped.query('iteration<=@iteration').groupby(['algorithm_name','problem_id','instance_id']).min()[str(dimension)].to_frame()
                minimums_up_to_iteration.columns=['min_up_to_iteration']
                minimums_up_to_iteration=minimums_up_to_iteration.reset_index()
                minimums_up_to_iteration['iteration']=iteration
                minimums_up_to_iteration['seed']=seed
                minimums_up_to_iteration['dimension']=dimension
                all_minimums_df=pd.concat([all_minimums_df, minimums_up_to_iteration])
            print(dimension,seed,algorithm_name)
            
all_minimums_df.to_csv('algorithm_run_data/all_algorithms_minimums_per_iteration.csv')

3 200 DE
3 200 ES
3 200 PSO
3 400 DE
3 400 ES
3 400 PSO
3 600 DE
3 600 ES
3 600 PSO
3 800 DE
3 800 ES
3 800 PSO
3 1000 DE
3 1000 ES
3 1000 PSO
5 200 DE
5 200 ES
5 200 PSO
5 400 DE


In [ ]:
all_minimums_df=pd.read_csv('algorithm_run_data/all_algorithms_minimums_per_iteration.csv',index_col=[0])

In [ ]:
all_bbob_optimums=pd.DataFrame()
for dimension in [3,5,10,20]:
    dimension_optimums=pd.read_csv(f'algorithm_run_data/BBOB_optimums_dim_{dimension}.csv', index_col=False)
    dimension_optimums['dimension']=dimension
    all_bbob_optimums=pd.concat([all_bbob_optimums, dimension_optimums])


In [ ]:
min_columns=all_minimums_df.columns

In [ ]:
all_minimums_optimums_df=all_minimums_df.merge(all_bbob_optimums, left_on=['problem_id', 'instance_id','dimension'], right_on=['problem_id', 'instance_id','dimension'])

In [ ]:
all_minimums_optimums_df=all_minimums_optimums_df[list(min_columns)+['y']]

In [ ]:
all_minimums_optimums_df['precision_at_iteration'] = all_minimums_optimums_df.apply(lambda row: row['min_up_to_iteration']-row['y'], axis=1)

In [ ]:
all_minimums_optimums_df.to_csv('algorithm_run_data/all_algorithms_precision_per_iteration.csv',compression='zip')